In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
import sys
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import train_test_split
from embedding_layers import SkeletalInputEmbedding
from encoder_layers import TransformerEncoder
from decoder_layers import TransformerDecoder
import TF_helper_functions as hf
import pickle
import numpy as np
import ipywidgets as widgets
from IPython.display import display


In [ ]:
data_path='/home/augustine/lfd_ws/src/skill_transfer/hmpar_former/data_process/process_data/'
# Base path and file information
file_root='24_08_30/'
file_name='training_raw_normalize.pkl'

model_path='/home/augustine/lfd_ws/src/skill_transfer/hmpar_former/models/'
model_name = model_path + '24_08_30_nor_v2.pth'

print(model_name)
def load_results_from_pickle(filename):
    with open(filename, 'rb') as f:
        print("load the file: ", filename)
        return pickle.load(f)
    

def process_all_datasets(results, input_length=60, predict_length=60):
    all_X_pos, all_X_vel, all_X_acc = [], [], []
    all_Y_pos, all_Y_vel, all_Y_acc = [], [], []
    discarded_frames = {}

    for i in range(1, int((len(results)-6)/3)+1):  # Assuming you have 2 datasets
        dataset_key = f'dataset{i}'
        norm_pos = results[f'{dataset_key}_normpos']
        norm_vel = results[f'{dataset_key}_normvel']
        norm_acc = results[f'{dataset_key}_normacc']
        # norm_pos = results[f'{dataset_key}_pos']
        # norm_vel = results[f'{dataset_key}_vel']
        # norm_acc = results[f'{dataset_key}_acc']

        # Generate sequences for this dataset
        X_pos, X_vel, X_acc, Y_pos, Y_vel, Y_acc = hf.generate_sequences(norm_pos, norm_vel, norm_acc, input_length, predict_length)
        
        all_X_pos.append(X_pos)
        all_X_vel.append(X_vel)
        all_X_acc.append(X_acc)
        all_Y_pos.append(Y_pos)
        all_Y_vel.append(Y_vel)
        all_Y_acc.append(Y_acc)

        # Calculate discarded frames
        total_frames = norm_pos.shape[0]
        used_frames = X_pos.shape[0] + input_length + predict_length - 1
        print(f"X_pos: {X_pos.shape[0]}, input_length: {input_length}, predict_length: {predict_length}")
        print(f"total_frames: {total_frames}, used_frames: {used_frames}")
        discarded = total_frames - used_frames
        discarded_frames[dataset_key] = discarded

    # Combine sequences from all datasets
    combined_X_pos = np.concatenate(all_X_pos)
    combined_X_vel = np.concatenate(all_X_vel)
    combined_X_acc = np.concatenate(all_X_acc)
    combined_Y_pos = np.concatenate(all_Y_pos)
    combined_Y_vel = np.concatenate(all_Y_vel)
    combined_Y_acc = np.concatenate(all_Y_acc)

    return (combined_X_pos, combined_X_vel, combined_X_acc, 
            combined_Y_pos, combined_Y_vel, combined_Y_acc, 
            discarded_frames)


In [ ]:
input_length = 30
predict_length = 2
datasetnum=6

# Load the results
results = load_results_from_pickle(data_path + file_root + file_name)
# Process all datasets and get combined sequences
(combined_X_pos, combined_X_vel, combined_X_acc, 
 combined_Y_pos, combined_Y_vel, combined_Y_acc, 
 discarded_frames) = process_all_datasets(results, input_length, predict_length)

print("Combined sequences shapes:")
print(f"X_pos shape: {combined_X_pos.shape}")
print(f"X_vel shape: {combined_X_vel.shape}")
print(f"X_acc shape: {combined_X_acc.shape}")
print(f"Y_pos shape: {combined_Y_pos.shape}")
print(f"Y_vel shape: {combined_Y_vel.shape}")
print(f"Y_acc shape: {combined_Y_acc.shape}")

print("\nDiscarded frames per dataset:")
for dataset, frames in discarded_frames.items():
    print(f"{dataset}: {frames} frames")


In [ ]:
# Assuming you have already defined your models and set up your training loop
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "CPU")

if torch.cuda.is_available():
    print("CUDA is available!")
    
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    # Get the name of each GPU and ensure the right one is being used
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        
    # Set and confirm the current device (if multiple GPUs are available)
    device = torch.device("cuda:0")  # You can change the index to use a different GPU
    print(f"Using GPU device: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")
    
# Initialize models
embed_dim = 128
num_heads = 8
num_layers = 6
num_joints = 6
dropout_rate = 0.1
dof=3
input_dim = num_joints * dof
batch_size = 1


start_token = torch.zeros(1, num_joints, 3).to(device)  # Assuming start token

# Assuming sequence length
# input_length = 30
# predict_length = 2


# tgt_mask= create_shifted_mask((input_length-1), num_joints)
# tgt_mask = tgt_mask.to(device)

# Convert to PyTorch tensors
X_pos_tensor = torch.tensor(combined_X_pos, dtype=torch.float32)
X_vel_tensor = torch.tensor(combined_X_vel, dtype=torch.float32)
X_acc_tensor = torch.tensor(combined_X_acc, dtype=torch.float32)

Y_pos_tensor = torch.tensor(combined_Y_pos, dtype=torch.float32)
Y_vel_tensor = torch.tensor(combined_Y_vel, dtype=torch.float32)
Y_acc_tensor = torch.tensor(combined_Y_acc, dtype=torch.float32)

# Create the full dataset
full_dataset = TensorDataset(X_pos_tensor, X_vel_tensor, X_acc_tensor, Y_pos_tensor, Y_vel_tensor, Y_acc_tensor)

# Data loader

train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=False, num_workers=16)

#Defining Models
embedding = SkeletalInputEmbedding(input_dim).to(device)
encoder = TransformerEncoder(embed_dim, num_heads, num_layers, dropout_rate).to(device)
decoder = TransformerDecoder(embed_dim, num_heads, num_layers, num_joints, dropout_rate).to(device)

# Loss function
#criterion = nn.MSELoss()
criterion = hf.MaskedMSELoss()

# Optimizer
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.00001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


# Training loop
num_epochs = 100
best_loss = float('inf')
train_losses = []

for epoch in range(num_epochs):
    encoder.train()
    decoder.train()
    total_loss = 0.0

    print(f'Epoch [{epoch}/{num_epochs}]')
    
    for batch in train_loader:
        # Unpack batch
        X_pos_batch, X_vel_batch, X_acc_batch, Y_pos_batch, Y_vel_batch, Y_acc_batch = batch
        
        # Memory Data
        X_pos_batch = X_pos_batch.to(device)
        X_vel_batch = X_vel_batch.to(device)
        X_acc_batch = X_acc_batch.to(device)

        inputembeddings = embedding(X_pos_batch, X_vel_batch)
       
        memory = encoder(inputembeddings, src_key_padding_mask=None)

        # Output Data
        Y_pos_batch = Y_pos_batch.to(device)
        Y_vel_batch = Y_vel_batch.to(device)
        Y_acc_batch = Y_acc_batch.to(device)

        #Target Data
        Y_pos_target=X_pos_batch[:, -1:, :, :]
        Y_vel_target=Y_vel_batch[:,:-1,:,:]
        Y_acc_target=Y_acc_batch[:,:-1,:,:]

        #Expected Data 
        
        Y_pos_expected=Y_pos_batch[:,1:,:,:]

        targetembeddings = embedding(Y_pos_target, Y_vel_target)

        # Perform forward pass through decoder
        output = decoder(targetembeddings, memory, tgt_key_padding_mask=None, memory_key_padding_mask=None)

        output = output.where(~torch.isnan(output), torch.zeros_like(output))
        Y_pos_expected = Y_pos_expected.where(~torch.isnan(Y_pos_expected), torch.zeros_like(Y_pos_expected))

                
        # print("input embeddings: ", np.shape(inputembeddings))
        # print("memory: ", np.shape(memory))
        # print("target: ", np.shape(targetembeddings))
        # print("output: ", np.shape(output))
        # print("Expected output: ", np.shape(Y_pos_expected))
        # Compute loss
        loss = criterion(output, Y_pos_expected)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate total loss for the epoch
        total_loss += loss.item()
        
    scheduler.step()
    # Calculate average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    # Print or log average loss for the epoch
    print(f'Loss: {avg_loss:.4f}')
    
    # Save model weights if the current epoch has the best loss
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save({
            'epoch': epoch,
            'embedding_state_dict': embedding.state_dict(),
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss
        }, model_name)
        print(f'Saved new best model with validation loss: {avg_loss:.4f}')


# Plot training loss
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss')
plt.show()
